<a href="https://colab.research.google.com/github/johnpharmd/LS-DS-Unit-1-Project/blob/master/LS_DS_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip

--2018-12-11 17:05:13--  https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3347213 (3.2M) [application/zip]
Saving to: ‘dataset_diabetes.zip’

dataset_diabetes.zi 100%[===================>]   3.19M  1.69MB/s    in 1.9s    

2018-12-11 17:05:20 (1.69 MB/s) - ‘dataset_diabetes.zip’ saved [3347213/3347213]



In [8]:
!unzip dataset_diabetes.zip

Archive:  dataset_diabetes.zip
  inflating: dataset_diabetes/diabetic_data.csv  
  inflating: dataset_diabetes/IDs_mapping.csv  


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [0]:
# Make the main and supplemental dataframes
df = pd.read_csv('dataset_diabetes/diabetic_data.csv')
icd9_df = pd.read_csv('https://raw.githubusercontent.com/drobbins/ICD9/master/icd9.txt', delimiter='\t', encoding='ISO-8859-1')

In [0]:
print(icd9_df.shape)
icd9_df.head()

(14567, 4)


,DIAGNOSIS CODE,LONG DESCRIPTION,SHORT DESCRIPTION,Unnamed: 3
0,0010,Cholera due to vibrio cholerae,Cholera d/t vib cholerae,NaN
1,0011,Cholera due to vibrio cholerae el tor,Cholera d/t vib el tor,NaN
2,0019,"Cholera, unspecified",Cholera NOS,NaN
3,0020,Typhoid fever,Typhoid fever,NaN
4,0021,Paratyphoid fever A,Paratyphoid fever a,NaN


In [18]:
icd9_df = icd9_df.drop('Unnamed: 3', axis=1)
icd9_df.columns

Index(['DIAGNOSIS CODE', 'LONG DESCRIPTION', 'SHORT DESCRIPTION'], dtype='object')

In [0]:
print(df.shape)
df.head()

(101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [0]:
# df.weight.value_counts()

In [0]:
# Drop 'weight' from df, given that ~96.9% of observations have '?'
# df = df.drop('weight', axis=1)
df.columns

In [17]:
# 'Citoglipton' does not appear to be a valid generic drug name for Januvia
df = df.rename(columns={'citoglipton': 'sitagliptin'})
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'sitagliptin', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [38]:
letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWYXZ'
nums = '0123456789'
df.isin(['?', '+', '-', (letter for letter in letters), (num for num in nums)]).sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide     

In [56]:
df.isin(['?', '+', '-', (letter for letter in letters)]).sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide     

In [68]:
pd.set_option('display.max_columns', None)
len(df.loc[(df.diag_1.isin(['?']) & (df.diag_2.isin(['?'])))])

1

In [55]:
df.isin([num for num in nums]).sum()

encounter_id                  0
patient_nbr                   0
race                          0
gender                        0
age                           0
admission_type_id             0
discharge_disposition_id      0
admission_source_id           0
time_in_hospital              0
payer_code                    0
medical_specialty             0
num_lab_procedures            0
num_procedures                0
num_medications               0
number_outpatient             0
number_emergency              0
number_inpatient              0
diag_1                      585
diag_2                      208
diag_3                      202
number_diagnoses              0
max_glu_serum                 0
A1Cresult                     0
metformin                     0
repaglinide                   0
nateglinide                   0
chlorpropamide                0
glimepiride                   0
acetohexamide                 0
glipizide                     0
glyburide                     0
tolbutam

In [69]:
# Method courtesy of Ryan Herr - LambdaSchool
for col in df.select_dtypes(['object']):
  single_character = df[col].str.len() == 1
  print(col, single_character.sum())

race 2273
gender 0
age 0
payer_code 40256
medical_specialty 49949
diag_1 606
diag_2 566
diag_3 1625
max_glu_serum 0
A1Cresult 0
metformin 0
repaglinide 0
nateglinide 0
chlorpropamide 0
glimepiride 0
acetohexamide 0
glipizide 0
glyburide 0
tolbutamide 0
pioglitazone 0
rosiglitazone 0
acarbose 0
miglitol 0
troglitazone 0
tolazamide 0
examide 0
sitagliptin 0
insulin 0
glyburide-metformin 0
glipizide-metformin 0
glimepiride-pioglitazone 0
metformin-rosiglitazone 0
metformin-pioglitazone 0
change 0
diabetesMed 0
readmitted 0


In [21]:
df.isna().sum().sum()

0

In [22]:
df.isnull().sum().sum()

0

In [29]:
df.applymap(np.isreal).sum()

encounter_id                101766
patient_nbr                 101766
race                             0
gender                           0
age                              0
admission_type_id           101766
discharge_disposition_id    101766
admission_source_id         101766
time_in_hospital            101766
payer_code                       0
medical_specialty                0
num_lab_procedures          101766
num_procedures              101766
num_medications             101766
number_outpatient           101766
number_emergency            101766
number_inpatient            101766
diag_1                           0
diag_2                           0
diag_3                           0
number_diagnoses            101766
max_glu_serum                    0
A1Cresult                        0
metformin                        0
repaglinide                      0
nateglinide                      0
chlorpropamide                   0
glimepiride                      0
acetohexamide       

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 49 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non-null int64
diag_1                      101766

In [32]:
df.race.value_counts()

Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

In [0]:
df.age.value_counts()

In [0]:
df.A1Cresult.value_counts()

None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64

In [0]:
df.metformin.value_counts()

No        81778
Steady    18346
Up         1067
Down        575
Name: metformin, dtype: int64

In [0]:
df.diag_1.value_counts()

In [0]:
# Compare how ICD-9 codes are constructed in each dataframe
print('in icd9_df:', [x for x in icd9_df['DIAGNOSIS CODE'] if '428' in x][:10])

print('in df:', [y for y in df.diag_1 if '428' in y][:10])

in icd9_df: ['1428', '24280', '24281', '34280', '34281', '34282', '4280', '4281', '42820', '42821']
in df: ['428', '428', '428', '428', '428', '428', '428', '428', '428', '428']


In [0]:
# icd9_df = icd9_df.applymap(lambda x: x.rstrip('.0'))
print('in icd9_df:', [x for x in icd9_df['DIAGNOSIS CODE'] if 'V2' in x][:10])

in icd9_df: ['V2', 'V201', 'V202', 'V2031', 'V2032', 'V21', 'V211', 'V212', 'V213', 'V2131']


In [0]:
icd9_df['DIAGNOSIS CODE'].value_counts()
icd9_df.isin(['.0']).sum()

DIAGNOSIS CODE       0
LONG DESCRIPTION     0
SHORT DESCRIPTION    0
dtype: int64

In [0]:
DX_CODE = icd9_df['DIAGNOSIS CODE']


In [51]:
DX_CODE.value_counts()

E8199    1
8512     1
E9918    1
0383     1
37001    1
25043    1
37919    1
7765     1
27403    1
56721    1
63572    1
2771     1
36801    1
78942    1
003      1
29513    1
7625     1
308      1
92232    1
3872     1
5324     1
E9936    1
1363     1
V213     1
9415     1
521      1
6516     1
8364     1
E0191    1
34121    1
        ..
V252     1
7352     1
71833    1
E958     1
9565     1
44502    1
86813    1
E9224    1
9982     1
9517     1
36963    1
V9089    1
80303    1
52573    1
9662     1
V029     1
V469     1
5232     1
5356     1
44772    1
34212    1
V5972    1
01235    1
37513    1
80518    1
40519    1
2503     1
79672    1
01361    1
V187     1
Name: DIAGNOSIS CODE, Length: 14567, dtype: int64

In [0]:
df.max_glu_serum.value_counts()

In [0]:
df.metformin.value_counts()

In [0]:
df.loc[(df.max_glu_serum != 'None') & (df.metformin != 'No')]